In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import torchvision
import torch.optim as optim
import argparse
import matplotlib
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from tqdm import tqdm
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import pandas as pd
import numpy as np
matplotlib.style.use('ggplot')
from sklearn.model_selection import train_test_split
import lightning.pytorch as pl
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.preprocessing import MinMaxScaler
import math

torch.manual_seed(33)

c:\Users\mdzak\anaconda3\envs\fyp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Import Data

In [2]:
dna_meth = pd.read_csv('new_clean_data/consistent_dna_meth.csv')
dna_meth.drop(columns='Unnamed: 0', inplace=True)
dna_meth

,CELL_LINE_NAME,1:134999,1:135191;1:135218,1:135203;1:135208,1:713376;1:713388;1:713400;1:713448;1:713450;1:713454,1:713901;1:713921;1:714178;1:714182;1:714199;1:714254;1:714258;1:714261;1:714264;1:714277;1:714278;1:714293;1:714301;1:714491;1:714511;1:714566;1:714584,1:715390;1:715392;1:715405;1:715415,1:804993;1:804999;1:805282;1:805284;1:805290;1:805327;1:805338;1:805341;1:805352;1:805445;1:805450;1:805467;1:805468,1:805474;1:805477;1:805479,1:805484;1:805486,...,Dactinomycin,Daporinad,Dasatinib,Rapamycin,Romidepsin,SN-38,Temsirolimus,Trametinib,Vinblastine,Vinorelbine
0,22rv1,0.9821,1.00000,1.00000,0.41803,0.00000,0.23610,0.00000,0.10000,0.0000,...,-4.840963,-3.112784,4.203067,-4.382138,-4.917572,-4.972312,-0.974396,1.325603,-4.384381,-3.401996
1,2313287,0.8897,0.80360,0.99105,0.65648,0.00000,0.24032,0.00000,0.00000,0.0000,...,-4.584971,-3.336795,1.797167,-3.486065,-6.017003,-4.132899,3.279539,-0.040150,-4.849422,-5.303091
2,42mgba,0.9222,0.87500,1.00000,0.49984,0.00897,0.95695,0.02262,0.12001,0.1765,...,-4.240733,-3.336795,-2.233603,-3.671158,-6.415517,-4.737156,-0.074308,1.621553,-4.784344,-3.927690
3,5637,1.0000,0.95915,1.00000,0.75370,0.00000,0.03540,0.07694,0.00000,0.0000,...,-4.321517,-3.336795,-1.829021,-0.116397,-5.559258,-4.537337,1.818908,-0.275098,-3.494500,-5.252216
4,639v,1.0000,1.00000,1.00000,0.70308,0.00000,0.16765,0.05868,0.16667,0.3000,...,-4.725271,-3.336795,1.391965,-1.822279,-4.955669,-6.639479,1.248546,1.437469,-3.986408,-3.138535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,wsudlcl2,0.7586,0.76473,1.00000,0.70492,0.00000,0.72915,0.00000,0.00000,0.0000,...,-5.120747,-6.196767,-1.463565,-5.903482,-5.240336,-4.959568,-5.974346,0.641326,-7.103903,-6.196508
539,yapc,0.9595,0.98215,1.00000,0.96413,0.00000,0.86138,0.00357,0.00000,0.0000,...,-1.932980,-5.076682,1.407784,1.214362,-3.897758,5.073883,1.195720,3.598733,-0.095525,2.307072
540,yh13,0.9211,1.00000,0.97220,0.59999,0.00000,0.94020,0.01222,0.00000,0.0000,...,-3.595530,-3.469021,-0.787238,-2.743620,-5.957548,-3.526360,-1.963500,1.153223,-5.412392,-4.207989
541,ykg1,0.8617,0.92310,0.93395,0.82007,0.00000,0.99590,0.00000,0.00000,0.0000,...,-5.262451,3.115970,0.597641,-0.763934,-5.240336,-3.225249,2.191202,0.407173,-3.494500,-2.601804


In [3]:
# get columns
cell_line_cols = ['CELL_LINE_NAME']
drug_resp_cols = ['Dactinomycin', 'Daporinad', 'Dasatinib',	'Rapamycin', 'Romidepsin', 'SN-38',	'Temsirolimus',	'Trametinib', 'Vinblastine', 'Vinorelbine']
dna_meth_cols = dna_meth.drop(columns=['CELL_LINE_NAME','Dactinomycin', 'Daporinad', 'Dasatinib',	'Rapamycin', 'Romidepsin', 'SN-38',	'Temsirolimus',	'Trametinib', 'Vinblastine', 'Vinorelbine']).columns

In [4]:
# drug response labels to tensor
drug_resp_labels = np.stack([dna_meth[col].values for col in drug_resp_cols], 1)
drug_resp_labels = torch.tensor(drug_resp_labels, dtype=torch.float)

In [5]:
# dna_meth values to tensor
dna_meth_values = np.stack([dna_meth[col].values for col in dna_meth_cols], 1)
dna_meth_values = torch.tensor(dna_meth_values, dtype=torch.float)

In [6]:
# cell line values to list
cell_line_list = []
for col in cell_line_cols:
    cell_line_list.append(dna_meth[col].values)

### Define Dataloaders

In [7]:
batch_size = 32

full_dataset = torch.utils.data.TensorDataset(dna_meth_values, drug_resp_labels)
full_loader = DataLoader(full_dataset, shuffle=False, batch_size=batch_size)

# dna_meth_train, dna_meth_test, drug_resp_labels_train, drug_resp_labels_test = train_test_split(dna_meth_values, drug_resp_labels, test_size= 0.1, random_state= 0, shuffle=False)

# train_dataset = torch.utils.data.TensorDataset(dna_meth_train, drug_resp_labels_train)
# test_dataset = torch.utils.data.TensorDataset(dna_meth_test, drug_resp_labels_test)

# train_loader = DataLoader(train_dataset, shuffle= False, batch_size= batch_size)
# test_loader = DataLoader(test_dataset, shuffle= False, batch_size= batch_size)

### VAE model with Regressor

In [8]:
class VAE_with_regressor(nn.Module):
    def __init__(self, input_size, level_2, level_3, latent_dim):
        super(VAE_with_regressor, self).__init__()
        cat_features = 10
        # Encoder layers
        self.enc_fc1 = nn.Sequential(
                        nn.Linear(input_size, level_2),
                        nn.BatchNorm1d(level_2),
                        nn.ReLU())
        
        self.enc_fc2 = nn.Sequential(
                        nn.Linear(level_2, level_3),
                        nn.BatchNorm1d(level_3),
                        nn.ReLU())

        self.enc_fc3_mean = nn.Sequential(
                    nn.Linear(level_3, latent_dim),
                    nn.BatchNorm1d(latent_dim))
        
        self.enc_fc3_log_var = nn.Sequential(
                    nn.Linear(level_3, latent_dim),
                    nn.BatchNorm1d(latent_dim))
        
        
        # Decoder layers
        self.dec_fc3 = nn.Sequential(
                        nn.Linear(latent_dim, level_3),
                        nn.BatchNorm1d(level_3),
                        nn.ReLU())
        
        self.dec_fc2 = nn.Sequential(
                        nn.Linear(level_3, level_2),
                        nn.BatchNorm1d(level_2),
                        nn.ReLU())
        
        self.dec_fc1 = nn.Sequential(
                    nn.Linear(level_2, input_size),
                    nn.BatchNorm1d(input_size),
                    nn.Sigmoid())
        
        # Regression fc layers
        self.r_fc1 = nn.Sequential(
                    nn.Linear(latent_dim, 64),
                    nn.BatchNorm1d(64),
                    nn.ReLU())
        self.r_fc2 = nn.Sequential(
                    nn.Linear(64, 32),
                    nn.BatchNorm1d(32),
                    nn.ReLU())
        
        self.r_fc3 = nn.Sequential(
                    nn.Linear(32, 10),
                    nn.BatchNorm1d(10))


    def encode(self, x):
        l2_layer = self.enc_fc1(x)
        l3_layer = self.enc_fc2(l2_layer)
        
        mu = self.enc_fc3_mean(l3_layer)
        logvar = self.enc_fc3_log_var(l3_layer)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def decode(self, z):
        l3_layer = self.dec_fc3(z)
        l2_layer = self.dec_fc2(l3_layer)
        x_hat = self.dec_fc1(l2_layer)
        return x_hat

    def regressor(self, mean):
        level_1_layer = self.r_fc1(mean)
        level_2_layer = self.r_fc2(level_1_layer)
        output_layer = self.r_fc3(level_2_layer)
        return output_layer
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_hat = self.decode(z)
        pred_y = self.regressor(mu)
        return x_hat, mu, logvar, z, pred_y

### Define loss functions

In [9]:
def vae_loss_function(x_hat, x, mean, log_var): # recon loss and kld loss
        bce = torch.nn.functional.binary_cross_entropy(x_hat, x, reduction = 'sum')
        kld = 0.5 * torch.sum(log_var.exp() + mean.pow(2) - 1 - log_var)
        loss = kld + bce
        return loss, kld, bce

In [10]:
def rmse_loss_function(y_pred, y):
    loss_fn = torch.nn.MSELoss(reduction='sum')
    loss = torch.sqrt(loss_fn(y_pred, y))
    return loss

In [11]:
def mse_loss_function(y_pred, y):
    loss_fn = torch.nn.MSELoss(reduction='sum')
    loss = loss_fn(y_pred, y)
    return loss

In [12]:
def mse_loss_function_mean(y_pred, y):
    loss_fn = torch.nn.MSELoss(reduction='mean')
    loss = loss_fn(y_pred, y)
    return loss

### Validation functions

In [13]:
def r_squared(y_true, y_pred):
    y_true_mean = torch.mean(y_true)
    SS_res = torch.sum(torch.square(y_true - y_pred))
    SS_tot = torch.sum(torch.square(y_true - y_true_mean))
    r_squared = 1 - SS_res / (SS_tot + torch.finfo(torch.float32).eps)
    return r_squared

### Training parameters

In [14]:
epochs = 50

lr = 0.001
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

k = 10
splits = KFold(n_splits=k, shuffle=False)

In [15]:
# Initialize the model
input_size = 81037 #dimension of gene expressions
level_2 = 2048
level_3 = 1024
latent_dim = 128 # target latent size
model = VAE_with_regressor(input_size, level_2, level_3, latent_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

### Train function

In [16]:
def train (model, dataloader, epoch, optimizer, w_recon_loss, w_kl_loss, w_reg_loss):
    model.train()
    train_recon = 0
    train_kl  = 0
    train_reg = 0
    train_total_loss = 0
    batch_num = 0

    for batch_idx, batch in enumerate(dataloader):
        batch_num = batch_num + 1
        # get data values
        dna_meth_values, drug_resp_labels = batch
        dna_meth_values = dna_meth_values.to(device)
        drug_resp_labels = drug_resp_labels.to(device)

        # clear gradients
        optimizer.zero_grad()
        # apply model
        x_hat, mu, logvar, z, pred_y = model(dna_meth_values)

        # losses
        vae_loss, kld, bce = vae_loss_function(x_hat, dna_meth_values, mu, logvar)
        mse_loss = mse_loss_function_mean(pred_y, drug_resp_labels)

        #apply loss weights
        loss = w_recon_loss * bce + w_kl_loss * kld + w_reg_loss * mse_loss

        

        # backward
        loss.backward()

        with torch.no_grad():
            train_recon += bce.item()
            train_kl += kld.item()
            train_reg += mse_loss.item()
            train_total_loss += loss.item()

        # update parameters
        optimizer.step()

    train_recon_ave = train_recon/ len(dataloader.sampler)
    train_kl_ave = train_kl/ len(dataloader.sampler)
    train_reg_ave = train_reg/ batch_num
    train_total_ave = train_total_loss/ len(dataloader.sampler)

    print('=====> Epoch {} \n' 
          'Average Recon Loss: {:.3f} \n'
          'Average KL Loss: {:.3f} \n'
          'Average MSE Loss: {:.3f} \n'.format(epoch, train_recon_ave, train_kl_ave, train_reg_ave))
    
    return train_recon_ave, train_kl_ave, train_reg_ave, train_total_ave


### Validation Function

In [17]:
def test(dataloader, model):
    model.eval()
    test_recon = 0
    test_kl  = 0
    test_reg = 0
    test_rmse = 0
    test_r2 = 0
    batch_num = 0
    with torch.no_grad():
        mean_store = torch.zeros(1, latent_dim).to(device)
        pred_store = torch.zeros(1,10).to(device)
        for batch_idx, batch in enumerate(dataloader):
            batch_num = batch_num + 1
            # get data values
            dna_meth_values, drug_resp_labels = batch
            dna_meth_values = dna_meth_values.to(device)
            drug_resp_labels = drug_resp_labels.to(device)

            # apply model
            x_hat, mu, logvar, z, pred_y = model(dna_meth_values)

            # loss functions
            vae_loss, kld, bce = vae_loss_function(x_hat, dna_meth_values, mu, logvar)
            mse_loss = mse_loss_function(pred_y, drug_resp_labels)
            r2_value = r_squared(drug_resp_labels, pred_y)

            test_recon += bce.item()
            test_kl += kld.item()
            test_reg += mse_loss.item()
            test_r2 += r2_value.item()

            # Get relevant values
            mean_store = torch.cat((mean_store, mu), 0)
            pred_store = torch.cat((pred_store, pred_y), 0)

        # ====== export reduced dimensions and predictions ======
        mean_final = mean_store[1:]
        mean_final_np = mean_final.cpu().numpy()
        mean_final_df = pd.DataFrame(mean_final_np)

        pred_final = pred_store[1:]
        pred_final_np = pred_final.cpu().numpy()
        pred_final_df = pd.DataFrame(pred_final_np)

        input_path = 'C:/Users/mdzak/Desktop/GitHub/FYP_Zaki/notebooks_new/output/'

        mean_path =  input_path + str(latent_dim) + 'D_dna_meth_latent_code.tsv'
        pred_path = input_path + str(latent_dim) + 'D_dna_meth_predictions.tsv'

        mean_final_df.to_csv(mean_path, sep='\t')
        pred_final_df.to_csv(pred_path, sep='\t')

        # print loss
        test_recon_ave = test_recon/ len(dataloader.sampler)
        test_kl_ave = test_kl/ len(dataloader.sampler)
        test_reg_ave = test_reg/ (len(dataloader.sampler) * 10)
        test_rmse_ave = math.sqrt(test_reg_ave)
        test_r2_ave = test_r2/ batch_num

        print('Average Recon Loss: {:.3f} \n'
          'Average KL Loss: {:.3f} \n'
          'Average MSE Loss: {:.3f} \n'
          'Average RMSE Loss: {:.3f} \n'
          'Average R2: {:.3f} \n'.format(test_recon_ave, test_kl_ave, test_reg_ave, test_rmse_ave, test_r2_ave))
        
        return test_recon_ave, test_kl_ave, test_reg_ave, test_rmse_ave, test_r2_ave

### Execute Training (Cross Validation)

In [18]:
history_train = {'Train Recon Loss': [], 'Train KL Loss': [], 'Train MSE Loss': [], 'Train Total Loss': []}
history_test = {'Test Recon Loss': [], 'Test KL Loss': [], 'Test MSE Loss': [], 'Test RMSE Loss': [], 'Test R2': []}

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(full_dataset)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(full_dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(full_dataset, batch_size=batch_size, sampler=test_sampler)

    print("Start Training (Unsupervised Phase)")
    for epoch in range(1, epochs + 1):
       train_recon_ave, train_kl_ave, train_reg_ave, train_total_ave = train(model,train_loader, epoch, optimizer, w_recon_loss=1, w_kl_loss=1, w_reg_loss=0)
       history_train['Train Recon Loss'].append(train_recon_ave)
       history_train['Train KL Loss'].append(train_kl_ave)
       history_train['Train MSE Loss'].append(train_reg_ave)
       history_train['Train Total Loss'].append(train_total_ave)

    print("Start Training (Supervised Phase)")
    for epoch in range(1, epochs + 1):
        train_recon_ave, train_kl_ave, train_reg_ave, train_total_ave = train(model,train_loader, epoch, optimizer, w_recon_loss=0, w_kl_loss=0, w_reg_loss=1)
        history_train['Train Recon Loss'].append(train_recon_ave)
        history_train['Train KL Loss'].append(train_kl_ave)
        history_train['Train MSE Loss'].append(train_reg_ave)
        history_train['Train Total Loss'].append(train_total_ave)

    test_recon_ave, test_kl_ave, test_reg_ave, test_rmse_ave, test_r2_ave = test(test_loader, model)
    history_test['Test Recon Loss'].append(test_recon_ave)
    history_test['Test KL Loss'].append(test_kl_ave)
    history_test['Test MSE Loss'].append(test_reg_ave)
    history_test['Test RMSE Loss'].append(test_rmse_ave)
    history_test['Test R2'].append(test_r2_ave)

Fold 1
Start Training (Unsupervised Phase)
=====> Epoch 1 
Average Recon Loss: 57831.100 
Average KL Loss: 113.556 
Average MSE Loss: 13.344 

=====> Epoch 2 
Average Recon Loss: 54712.327 
Average KL Loss: 108.740 
Average MSE Loss: 13.619 

=====> Epoch 3 
Average Recon Loss: 53732.118 
Average KL Loss: 104.840 
Average MSE Loss: 13.353 

=====> Epoch 4 
Average Recon Loss: 53263.044 
Average KL Loss: 101.137 
Average MSE Loss: 13.592 

=====> Epoch 5 
Average Recon Loss: 52748.168 
Average KL Loss: 99.768 
Average MSE Loss: 13.588 

=====> Epoch 6 
Average Recon Loss: 52295.067 
Average KL Loss: 98.110 
Average MSE Loss: 13.412 

=====> Epoch 7 
Average Recon Loss: 51696.341 
Average KL Loss: 96.494 
Average MSE Loss: 13.422 

=====> Epoch 8 
Average Recon Loss: 51331.447 
Average KL Loss: 96.137 
Average MSE Loss: 13.342 

=====> Epoch 9 
Average Recon Loss: 50883.970 
Average KL Loss: 95.617 
Average MSE Loss: 13.370 

=====> Epoch 10 
Average Recon Loss: 50508.332 
Average KL Los

In [19]:
# print("Start Training (Unsupervised Phase)")
# for epoch in range(1, epochs + 1):
#     train(model,train_loader, epoch, optimizer, w_vae_loss=1, w_reg_loss=0)

In [20]:
# print("Start Training (Supervised Phase)")
# for epoch in range(1, epochs + 1):
#     train(model,train_loader, epoch, optimizer, w_vae_loss=0, w_reg_loss=1)

### Performance of CV

In [1]:
train_avg_cv_recon_loss = np.mean(history_train['Train Recon Loss'])
train_avg_cv_kl_loss = np.mean(history_train['Train KL Loss'])
train_avg_cv_mse_loss = np.mean(history_train['Train MSE Loss'])
train_avg_cv_total_loss = np.mean(history_train['Train Total Loss'])

test_avg_recon_loss =  np.mean(history_test['Test Recon Loss'])
test_avg_kl_loss =  np.mean(history_test['Test KL Loss'])
test_avg_mse_loss =  np.mean(history_test['Test MSE Loss'])
test_avg_rmse_loss =  np.mean(history_test['Test RMSE Loss'])
test_avg_r2_loss =  np.mean(history_test['Test R2'])

print('Performance of {} fold cross validation {} dimension:'.format(k, latent_dim))
print("\nAverage Training Recon Loss: {:.4f} \n Average Training KL Loss: {:.4f} \n Average Training RMSE Loss: {:.3f} \n Average Training Total Loss: {:.3f} \n".format(train_avg_cv_recon_loss, train_avg_cv_kl_loss, train_avg_cv_mse_loss, train_avg_cv_total_loss))
print("\nAverage Testing Recon Loss: {:.4f} \n Average Testing KL Loss: {:.4f} \n Average Testing MSE Loss: {:.3f} \n Average Testing RMSE Loss: {:.3f} \n Average Testing R2 Loss: {:.3f} \n".format(test_avg_recon_loss, test_avg_kl_loss, test_avg_mse_loss, test_avg_rmse_loss, test_avg_r2_loss))  


NameError: name 'np' is not defined

In [22]:
np.mean(history_test['Test RMSE Loss'])

1.674289744094556

In [23]:
history_test['Test RMSE Loss']

[2.849237377095305,
 2.254316064346069,
 1.828225612761639,
 1.6366155399814923,
 1.559920722707668,
 1.3816201260239909,
 1.4188095540851018,
 1.258016153069804,
 1.2665763331989643,
 1.289559957675526]

In [24]:
history_test['Test MSE Loss']

[8.118153631036932,
 5.08194091796875,
 3.3424088911576706,
 2.678510425708912,
 2.4333526611328127,
 1.9088741726345486,
 2.0130205507631653,
 1.5826046413845487,
 1.6042156078197338,
 1.6629648844401042]

### Save model

In [25]:
# torch.save(model.state_dict(), 'trained_models/VAE_dna_meth' + str(latent_dim) + '.pt')